<a href="https://colab.research.google.com/github/chw8207/Transformer/blob/main/%EB%8B%A4%EC%9A%B4%EC%8A%A4%ED%8A%B8%EB%A6%BCNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import tensorflow as tf
import pandas as pd
import numpy
from transformers import BertForSequenceClassification
from transformers import pipeline
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

### gpu 설정

In [7]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0' :
    raise SystemError('GPU device not found')
print(f'Found GPU at: {device_name}')

Found GPU at: /device:GPU:0


In [8]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16608443771653246284
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14328594432
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9970639381095083078
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


### 트랜스포머, torch 설정

In [9]:
try :
  import transformers
except :
  print('Installing transformers')
  !pip -qq install transformers

try :
  import torch
except :
  print('Installing Torch')
  !pip -qq install torch

### 다운스트림 작업 실행

#### CoLA(언어 수용성 코퍼스)

In [19]:
# 데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Transformer/data/in_domain_train.tsv',
                 delimiter='\t', header=None,
                 names=['sentence_source','label','label_notes','sentence'])

In [20]:
df.shape

(8551, 4)

In [23]:
# 사전 훈련된 BERT모델 로드하기
model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/Colab Notebooks/Transformer/bert-base-uncased',
                                                      num_labels=2)

Some weights of the model checkpoint at /content/drive/MyDrive/Colab Notebooks/Transformer/bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the mo

#### SST-2(스탠포드 감정 트리뱅크)

In [25]:
nlp = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [26]:
print(nlp('If you somtimes like to go to the movies to have fun,\
Wasabi is a good place to start.'),'If you sometimes like to go to the movies \
to have fun, Wasabi is a good place to start.')
print(nlp('Effective but too-tepid biopic.'),'Effective but too-tepid \
biopic')

[{'label': 'POSITIVE', 'score': 0.9997747540473938}] If you sometimes like to go to the movies to have fun, Wasabi is a good place to start.
[{'label': 'NEGATIVE', 'score': 0.9974064230918884}] Effective but too-tepid biopic


#### MRPC(Microsoft Research Paraphrase Corpus)
- 웹의 새로운 소스에서 추출한 문장 쌍들이 들어 있음

In [43]:
# paraphrase classification
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased-finetuned-mrpc')
model = TFAutoModelForSequenceClassification.from_pretrained('bert-base-cased-finetuned-mrpc')

classes = ['not paraphrase', 'is paraphrase']

sequence_A = 'The DVD-CCA then appealed to the state Supreme Court.'
sequence_B = 'The DVD-CCA appealed that decision to the U.S. Supreme Court.'

# tokenizer.encode_plus() : 시퀀스의 특수 토큰을 추가하고 필요한 경우 패딩 수행
paraphrase = tokenizer.encode_plus(sequence_A, sequence_B, return_tensors='tf')

paraphrase_classification_logits = model(paraphrase)[0]

paraphrase_results = tf.nn.softmax(paraphrase_classification_logits, axis=1).numpy()[0]

print(sequence_B, 'should be a paraphrase')
for i in range(len(classes)) :
  print(f'{classes[i]}: {round(paraphrase_results[i]*100)}%')

Some layers from the model checkpoint at bert-base-cased-finetuned-mrpc were not used when initializing TFBertForSequenceClassification: ['dropout_183']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at bert-base-cased-finetuned-mrpc.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


The DVD-CCA appealed that decision to the U.S. Supreme Court. should be a paraphrase
not paraphrase: 8%
is paraphrase: 92%
